In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlpa').getOrCreate()

In [0]:
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer, StandardScaler, Tokenizer, RegexTokenizer, StopWordsRemover, NGram, HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator, RegressionEvaluator
from pyspark.sql.functions import year, month, dayofmonth, col, udf
from pyspark.sql.types import IntegerType
from pyspark.ml.clustering import KMeans
from pyspark.ml.recommendation import ALS

In [0]:
sen_df = spark.createDataFrame([
  (0,'Hi I heard oabout spark'),
  (1,'I wish java could use case classes'),
  (2,'Logistic,regfression,models,are,neat')
], ['id','sentence'])

In [0]:
sen_df.show()

+---+--------------------+
 id| sentence|
+---+--------------------+
 0|Hi I heard oabout...|
 1|I wish java could...|
 2|Logistic,regfress...|
+---+--------------------+

In [0]:
tzr = Tokenizer(inputCol='sentence', outputCol='words')

In [0]:
rgxtzr = RegexTokenizer(inputCol='sentence', outputCol='words',pattern='\\W')

In [0]:
count_tokens = udf(lambda words:len(words),IntegerType())

In [0]:
tzd = tzr.transform(sen_df)
tzd.show()

+---+--------------------+--------------------+
 id| sentence| words|
+---+--------------------+--------------------+
 0|Hi I heard oabout...|[hi, i, heard, oa...|
 1|I wish java could...|[i, wish, java, c...|
 2|Logistic,regfress...|[logistic,regfres...|
+---+--------------------+--------------------+

In [0]:
tzd.withColumn('tokens',count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
 id| sentence| words|tokens|
+---+--------------------+--------------------+------+
 0|Hi I heard oabout...|[hi, i, heard, oa...| 5|
 1|I wish java could...|[i, wish, java, c...| 7|
 2|Logistic,regfress...|[logistic,regfres...| 1|
+---+--------------------+--------------------+------+

In [0]:
rgtzd = rgxtzr.transform(sen_df)
rgtzd.show()

+---+--------------------+--------------------+
 id| sentence| words|
+---+--------------------+--------------------+
 0|Hi I heard oabout...|[hi, i, heard, oa...|
 1|I wish java could...|[i, wish, java, c...|
 2|Logistic,regfress...|[logistic, regfre...|
+---+--------------------+--------------------+

In [0]:
rgtzd.withColumn('tokens',count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
 id| sentence| words|tokens|
+---+--------------------+--------------------+------+
 0|Hi I heard oabout...|[hi, i, heard, oa...| 5|
 1|I wish java could...|[i, wish, java, c...| 7|
 2|Logistic,regfress...|[logistic, regfre...| 5|
+---+--------------------+--------------------+------+

In [0]:
df = spark.createDataFrame([
  (0,['I','saw','the','green','horse']),
  (1,['Marry','had','a','little','lamb'])
], ['id','tokens'])
df.show()

+---+--------------------+
 id| tokens|
+---+--------------------+
 0|[I, saw, the, gre...|
 1|[Marry, had, a, l...|
+---+--------------------+

In [0]:
remover = StopWordsRemover(inputCol='tokens',outputCol='filtered')

In [0]:
remover.transform(df).show()

+---+--------------------+--------------------+
 id| tokens| filtered|
+---+--------------------+--------------------+
 0|[I, saw, the, gre...| [saw, green, horse]|
 1|[Marry, had, a, l...|[Marry, little, l...|
+---+--------------------+--------------------+

In [0]:
wdf = spark.createDataFrame([
    (0, ["Hi", "I", "heard", "about", "Spark"]),
    (1, ["I", "wish", "Java", "could", "use", "case", "classes"]),
    (2, ["Logistic", "regression", "models", "are", "neat"])
], ["id", "words"])
wdf.show()

+---+--------------------+
 id| words|
+---+--------------------+
 0|[Hi, I, heard, ab...|
 1|[I, wish, Java, c...|
 2|[Logistic, regres...|
+---+--------------------+

In [0]:
ngram = NGram(n=2,inputCol='words',outputCol='grams')

In [0]:
ngram.transform(wdf).select('grams').show(truncate=False)

+------------------------------------------------------------------+
grams |
+------------------------------------------------------------------+
[Hi I, I heard, heard about, about Spark] |
[I wish, wish Java, Java could, could use, use case, case classes]|
[Logistic regression, regression models, models are, are neat] |
+------------------------------------------------------------------+

In [0]:
sd = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic regression models are neat")
], ["label", "sentence"])

sd.show()

+-----+--------------------+
label| sentence|
+-----+--------------------+
 0.0|Hi I heard about ...|
 0.0|I wish Java could...|
 1.0|Logistic regressi...|
+-----+--------------------+

In [0]:
tzr = Tokenizer(inputCol='sentence', outputCol='words')

In [0]:
words_data = tzr.transform(sd)
words_data.show()

+-----+--------------------+--------------------+
label| sentence| words|
+-----+--------------------+--------------------+
 0.0|Hi I heard about ...|[hi, i, heard, ab...|
 0.0|I wish Java could...|[i, wish, java, c...|
 1.0|Logistic regressi...|[logistic, regres...|
+-----+--------------------+--------------------+

In [0]:
hashing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')

In [0]:
featurized_data = hashing_tf.transform(words_data)

In [0]:
idf = IDF(inputCol='rawFeatures', outputCol='features')

In [0]:
idf_model=idf.fit(featurized_data)

In [0]:

rescaled_data = idf_model.transform(featurized_data)
rescaled_data.show()

+-----+--------------------+--------------------+--------------------+--------------------+
label| sentence| words| rawFeatures| features|
+-----+--------------------+--------------------+--------------------+--------------------+
 0.0|Hi I heard about ...|[hi, i, heard, ab...|(262144,[18700,19...|(262144,[18700,19...|
 0.0|I wish Java could...|[i, wish, java, c...|(262144,[19036,20...|(262144,[19036,20...|
 1.0|Logistic regressi...|[logistic, regres...|(262144,[46243,58...|(262144,[46243,58...|
+-----+--------------------+--------------------+--------------------+--------------------+

In [0]:
dfb= spark.createDataFrame([
    (0, "a b c".split(" ")),
    (1, "a b b c a".split(" "))
], ["id", "words"])

In [0]:
cv = CountVectorizer(inputCol="words", outputCol="features", vocabSize=3, minDF=2.0)

In [0]:
model = cv.fit(dfb)

In [0]:
res = model.transform(dfb)

In [0]:
res.show(truncate=False)

+---+---------------+-------------------------+
id |words |features |
+---+---------------+-------------------------+
0 |[a, b, c] |(3,[0,1,2],[1.0,1.0,1.0])|
1 |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+